In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_csv("./melb_data.csv", index_col=0)

data.columns


Index(['Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude',
       'Regionname', 'Propertycount'],
      dtype='object')

In [19]:
y = data["Price"]
predictors = data.drop(["Price"], axis=1)
X = predictors.select_dtypes(exclude=["object"])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    mea = mean_absolute_error(predictions, y_valid)
    return mea

In [21]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

In [22]:
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)

183550.22137772635

In [23]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)

178166.46269899711

In [26]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col + "_was_missing"] = X_train_plus[col].isnull()
    X_valid_plus[col + "_was_missing"] = X_valid_plus[col].isnull()

my_imputer = SimpleImputer()

imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid)

178927.503183954

In [28]:
print(X_train.shape)

(10864, 12)


In [32]:
missing_val_count_by_col = (X_train.isnull().sum())
print(missing_val_count_by_col)


Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64


In [33]:
print(missing_val_count_by_col[missing_val_count_by_col>0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
